Из [статьи](https://www.researchgate.net/publication/332242485_Angular_Softmax_Loss_for_End-to-end_Speaker_Verification) "angular softmax speaker verification" и еще [одной статьи](https://deepai.org/publication/deep-speaker-embeddings-for-far-field-speaker-recognition-on-short-utterances) "Deep Speaker Embeddings for Speaker Recognition" взяты параметры для достижения цели этого проекта:
    features: MFFC - n_mel = 23, frame_length = 25ms, duration treck = 3s.
    loss: AMSoftmax(s=3,m-0.35), ArcFace loss, ASoftmax(m = 3)
    backend: cosine similarity

Use layer TimeDistributed from [article](https://ru-keras.com/wrappers-sloi/)

In [1]:
#libs
import tensorflow as tf
import numpy as np
import librosa
import os
import torch

from loss_layers import AMSoftmax, ArcFace
from pathlib import Path
from natsort import natsorted
from sklearn.model_selection import train_test_split
from DataGen import DataGenerator
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Conv1D, Conv2D, LSTM, Dropout,LeakyReLU, PReLU, Reshape, Bidirectional, TimeDistributed
from tensorflow.keras.layers import MaxPooling1D, MaxPooling2D, Flatten, concatenate, GlobalAveragePooling1D, AveragePooling1D
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers, initializers

In [2]:
path = Path('C:\\datasets_from_D\\train_vox2_2half\\wav_vad\\0\\')
all_audio = natsorted(list(path.rglob('*.wav')))
# print(audio[:150])
train_audio, test_audio = train_test_split(all_audio, test_size=0.2, random_state=42, shuffle=True)
print(len(train_audio), len(test_audio))

37776 9445


Prepaire generic data for train

In [3]:
# data = DataGenerator(train_audio[:20])
# d, cl = next(iter(data))
# print(d.shape, cl)
train_gen = DataGenerator(train_audio)
val_gen = DataGenerator(test_audio)

## Model from article "angular softmax speaker verification"


In [14]:
#Если не  с работает: 1 -  применить рекурентные слои, 2 - применить Conv2D, 3 - изменить Amsoftmax layer на Arclayer, 4- Применить amsoftmax loss а не слой.
from tensorflow.keras import regularizers
def create(input_shape=(301, 23)):
    input_1 = Input(shape=input_shape)


    x = Conv1D(23, kernel_size=5, activation='relu', kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4), name='Conv_1')(input_1)
    x = MaxPooling1D(pool_size=2, name='MP_1')(x)
    # x = Conv1D(128, kernel_size=3, activation='relu')(x)
    #
    # x = Conv1D(256, kernel_size=3, activation='relu')(x)
    # x = LSTM(128, return_sequences=True)(x)
    x = TimeDistributed(Dense(512, activation='relu', name="TD_1"))(x)
    x = BatchNormalization(momentum=0.99, name='BN_1')(x)
    x = Conv1D(512, 3, activation='relu', kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4), name='Conv_2')(x)
    x = MaxPooling1D(pool_size=2, name='MP_2')(x)
    x = TimeDistributed(Dense(512, activation='relu', name="TD_2"))(x)
    x = BatchNormalization(momentum=0.95, name='BN_2')(x)
    x = Conv1D(512, 3, activation='relu', kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4), name='Conv_3')(x)
    x = MaxPooling1D(pool_size=2, name='MP_3')(x)
    x = TimeDistributed(Dense(512, activation='relu', name="TD_3"))(x)
    x = BatchNormalization(momentum=0.99, name='BN_3')(x)
    x = Conv1D(512, 1, activation='relu', kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4), name='Conv_4')(x)
    x = MaxPooling1D(pool_size=2, name='MP_4')(x)
    x = TimeDistributed(Dense(512, activation='relu', name="TD_4"))(x)
    x = BatchNormalization(momentum=0.95, name='BN_4')(x)
    x = Conv1D(1500, 1, activation='relu', kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4), name='Conv_5')(x)
    x = TimeDistributed(Dense(1500, activation='relu', name="TD_5"))(x)
    x = BatchNormalization(momentum=0.99, name='BN_5')(x)
    x = AveragePooling1D(name='AVGP')(x)
    x = Flatten(name='Flt')(x)
    x = Dropout(0.5, name='Drop_1')(x)
    x = Dense(3000, activation='relu', kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4), name='D_1')(x)
    x = BatchNormalization(momentum=0.95, name='BN_6')(x)
    x = Dense(512, activation='relu', use_bias=False, kernel_regularizer=regularizers.L2(l2=1e-4), name='D_2')(x)
    embed = Dense(300, activation='relu', name='embed', use_bias=False, kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4))(x)
    outputs = AMSoftmax(1000, s=35, m=0.63, kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4), name='AMSoftmax')(embed)

    model = Model(input_1, outputs)
    model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.01), metrics=['accuracy'])

    return model

model = create()
model.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 301, 23)]         0         
                                                                 
 Conv_1 (Conv1D)             (None, 297, 23)           2668      
                                                                 
 MP_1 (MaxPooling1D)         (None, 148, 23)           0         
                                                                 
 time_distributed_30 (TimeDi  (None, 148, 512)         12288     
 stributed)                                                      
                                                                 
 BN_1 (BatchNormalization)   (None, 148, 512)          2048      
                                                                 
 Conv_2 (Conv1D)             (None, 146, 512)          786944    
                                                           

In [5]:
#Example TimeDistribute
# inputs = tf.keras.Input(shape=(10, 128, 128, 3))
# conv_2d_layer = tf.keras.layers.Conv2D(64, (3, 3))
# outputs = tf.keras.layers.TimeDistributed(conv_2d_layer)(inputs)
# outputs.shape


**Callbaxk**

In [15]:
sv_mod = ModelCheckpoint(
    filepath = 'D:\\Andrey\\data\\weights\\ARC_mffc_img\\\
model_TDL\\modelTD_b128_301_23_0f_ohe_{epoch:02d}-loss-{loss:.4f}_val_los-{val_loss:.4f}.hdf5',
    monitor = 'val_loss',
    mode = 'min',
    save_best_only = True
)
learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss',
                                            patience=6,
                                            verbose=1,
                                            factor=0.9,
                                            min_lr=0.00001)

**train**

In [ ]:
history = model.fit(train_gen,
              epochs = 100,
              validation_data = (val_gen),
              callbacks = [sv_mod, learning_rate_reduction],
              verbose = 1)

Epoch 1/100
2276/2276 [==============================] - 629s 275ms/step - loss: 7.9872 - accuracy: 0.0046 - val_loss: 7.4668 - val_accuracy: 0.0015 - lr: 0.0100
Epoch 2/100
2276/2276 [==============================] - 592s 260ms/step - loss: 7.0156 - accuracy: 0.0079 - val_loss: 7.7422 - val_accuracy: 9.3695e-04 - lr: 0.0100
Epoch 3/100
2276/2276 [==============================] - ETA: 0s - loss: 7.0037 - accuracy: 0.0088

In [9]:
#model.save('D:\Andrey\data\weights\ARC_mffc_img\model_TDL\Model_weights.h5')

In [10]:
model.load_weights('D:\Andrey\data\weights\ARC_mffc_img\model_TDL\Model_weights.h5')